This Colab Notebook was the basis for all our training with YoloV8. 

# Important: 

As we were not able to produce a good enough size distribution plot to compare our distributions with gruond truth, we will be using the following metrics for our evaluation: 

- Precision is the number of true positive detections (correctly detected craters) divided by the sum of true positive and false positive detections (all detections that were claimed as craters). It measures the accuracy of positive detections.

- Recall: Recall is the number of true positive detections divided by the sum of true positive and false negative detections (craters that were not detected). It measures the completeness of the detections.

- mAP50: mAP50 (mean Average Precision at IoU threshold of 0.5) is the average precision score over all craters, computed at an Intersection over Union (IoU) threshold of 0.5. It measures the performance of the object detector in terms of precision and recall.

- F1 Score: The F1 score is the harmonic mean of precision and recall. It is a single score that balances the trade-off between precision and recall and is commonly used as a single performance metric for binary classification problems.

# Imports: 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install ultralytics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!git clone https://github.com/ultralytics/ultralytics
%pip install -qe ultralytics

fatal: destination path 'ultralytics' already exists and is not an empty directory.
  Preparing metadata (setup.py) ... done


In [ ]:
pip install YOLO  

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
%cd /content/drive/MyDrive/yolo8/ul/ultralytics/yolo/engine

/content/drive/MyDrive/yolo8/ul/ultralytics/yolo/engine


In [ ]:
from model import YOLO


# Training YoloV8 on the Mars Data: 
- Compared to the Yolo5 with same paremeters,  Yolo8 performed better with high recall and precision, as well as high mAP50. 
- The yolo8n has the minimum parameters compared to other yolo8 model and it is the quickesk one to train. 
- Because of the Trade off between performance and running time, we choose Yolo8n as our best model for the mars dataset. Even when compared to YoloV5. 

**Yolo8 compared to Yolo5**
- Anchor-free detections are faster and more accurate than the previous yolo model. And this is the main difference in model architecture. 
- New backbone network, a new anchor-free detection head, and a new loss function making things much faster. 

**Our hyperparameter strategy**
- We changed the IOU to 0.5 to maximize the R and P, as well as mAP50 and therefore F1.
- We change the learning rate range to achieve a reasonable step size at each iteration while moving toward a minimum of the loss function.

In [ ]:
model = YOLO("yolov8n.pt") # pass any model type
model.train(data='/content/drive/MyDrive/yolo8/ul/ultralytics/yolo/data/marsnew.yaml', epochs=100, iou=0.5, lr0=0.001, lrf=0.2)

Ultralytics YOLOv8.0.26 🚀 Python-3.8.10 torch-1.13.1+cu116 CUDA:0 (NVIDIA A100-SXM4-40GB, 40536MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=/content/drive/MyDrive/yolo8/ul/ultralytics/yolo/data/marsnew.yaml, epochs=100, patience=50, batch=16, imgsz=640, save=True, cache=False, device=, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, overlap_mask=True, mask_ratio=4, dropout=False, val=True, save_json=False, save_hybrid=False, conf=0.001, iou=0.5, max_det=300, half=False, dnn=False, plots=True, source=ultralytics/assets/, show=False, save_txt=False, save_conf=False, save_crop=False, hide_labels=False, hide_conf=False, vid_stride=1, line_thickness=3, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimiz

In [ ]:
model = YOLO("yolov8n.pt")  # load an official model
model = YOLO("/content/ultralytics/runs/detect/train2/weights/best.pt")  # load a custom model

# Validate the model
results = model.val() 

Ultralytics YOLOv8.0.26 🚀 Python-3.8.10 torch-1.13.1+cu116 CUDA:0 (NVIDIA A100-SXM4-40GB, 40536MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs
val: Scanning /content/drive/MyDrive/yolov5/datasets/mars_sorted/labels/val.cache... 80 images, 0 backgrounds, 0 corrupt: 100%|██████████| 80/80 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:02<00:00,  1.90it/s]
                   all         80        142      0.914      0.923       0.96      0.554
Speed: 4.6ms pre-process, 2.9ms inference, 0.0ms loss, 1.1ms post-process per image


We wanted to implement this with the lunar data, but had a hard time with time managment so we will stick with the best model we have currently. That is, the ensemble model with left and right sides. 